# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
547,547,matamey,13.4231,8.4749,16.70,14,3,4.18,NE,1705897191
548,548,qujing,25.4833,103.7833,-0.62,88,100,4.92,CN,1705897191
549,549,qamdo,31.1667,97.2333,-8.13,41,18,1.28,CN,1705897192
550,550,kesinga,20.2000,83.2333,23.45,68,76,2.14,IN,1705897192
551,551,wanxian,30.8039,108.3897,5.99,74,100,2.54,CN,1705897192


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

lat_lngs = []
cities = []

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoLight",
    frame_width = 1000,
    frame_height = 700,
    size = "Humidity",
    scale = 0.7,
    color="City",
    alpha= 1
)

# Display the map
map_plot_1

 


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

#2-A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,east london,-33.0153,27.9116,21.52,83,0,3.60,ZA,1705896963
186,186,acapulco de juarez,16.8634,-99.8901,26.90,89,0,2.06,MX,1705897020
408,408,alo,7.4178,6.7371,22.30,21,0,1.28,NG,1705897128
536,536,nieuw amsterdam,5.8833,-55.0833,23.09,81,0,4.25,SR,1705897186
539,539,takoradi,4.8845,-1.7554,26.49,88,0,1.97,GH,1705897187
546,546,coahuayana de hidalgo,18.7000,-103.6583,22.62,80,0,1.55,MX,1705897191


In [18]:
r, c = city_data_df.shape
r, c

(6, 10)

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,east london,ZA,-33.0153,27.9116,83,
186,acapulco de juarez,MX,16.8634,-99.8901,89,
408,alo,NG,7.4178,6.7371,21,
536,nieuw amsterdam,SR,5.8833,-55.0833,81,
539,takoradi,GH,4.8845,-1.7554,88,
546,coahuayana de hidalgo,MX,18.7000,-103.6583,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation hotel',
          'apiKey': geoapify_key
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng'] 
    latitude = row['Lat']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"  


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
alo - nearest hotel: No hotel found
nieuw amsterdam - nearest hotel: No hotel found
takoradi - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
55,east london,ZA,-33.0153,27.9116,83,No hotel found
186,acapulco de juarez,MX,16.8634,-99.8901,89,No hotel found
408,alo,NG,7.4178,6.7371,21,No hotel found
536,nieuw amsterdam,SR,5.8833,-55.0833,81,No hotel found
539,takoradi,GH,4.8845,-1.7554,88,No hotel found
546,coahuayana de hidalgo,MX,18.7000,-103.6583,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 1,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)